In [135]:
import os
import re
import numpy as np
import pandas as pd
from PIL import Image
from sklearn import metrics
from skimage.io import imread
from bs4 import BeautifulSoup
from skimage.feature import hog
from skimage import io, transform
from sklearn import preprocessing
from skimage.color import rgb2gray
from skimage.transform import resize
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

!pip install lxml

In [136]:
l = []
# iterate over the subfolders in the train data folder
for root, dirs, files in os.walk( r'\Users\Family\Downloads\archive\images'):
     for file_name in files:
        # check if the file is a text file
        if file_name.endswith('.png'):
           file_path = os.path.join(root, file_name)
           # read the image data
           img = imread(file_path)
           # resize the image
           resized_img = resize(img, (128, 64))
           # reshape the image to match the expected format of hog
           hog_img = resized_img.reshape((resized_img.shape[0], resized_img.shape[1], -1))
           # calculate the HOG features
           #creating hog features 
           fd, hog_image = hog(hog_img, orientations=9, pixels_per_cell=(8, 8), 
                cells_per_block=(2, 2), visualize=True,multichannel=True)
           l.append(fd)

In [137]:
x = []
y = []
i = 0
# iterate over the subfolders in the train data folder
for root, dirs, files in os.walk( r'\Users\Family\Downloads\archive\annotations'):
     for file_name in files:
        # Reading the data inside the xml
        # file to a variable under the name
        # data
        file_path = os.path.join(root, file_name)
        with open(file_path, 'r') as f:
            data = f.read()
        # Passing the stored data inside
        # the beautifulsoup parser, storing
        # the returned object
        Bs_data = BeautifulSoup(data, "xml")
        # Finding all instances of tag
        # `unique`
        b_unique = Bs_data.find_all('name')
        xmin = Bs_data.find_all('xmin')
        ymin = Bs_data.find_all('ymin')
        xmax = Bs_data.find_all('xmax')
        ymax = Bs_data.find_all('ymax')
        
        for j in range(0,len(b_unique)):
            
            match1 = re.search(r"<xmin>(.*?)</xmin>", str(xmin[j]))
            match2 = re.search(r"<ymin>(.*?)</ymin>", str(ymin[j]))
            match3 = re.search(r"<xmax>(.*?)</xmax>", str(xmax[j]))
            match4 = re.search(r"<ymax>(.*?)</ymax>", str(ymax[j]))
            match5 = re.search(r"<name>(.*?)</name>", str(b_unique[j])) 
            
            body_of_tag1 = match1.group(1)
            body_of_tag2 = match2.group(1)
            body_of_tag3 = match3.group(1)
            body_of_tag4 = match4.group(1)
            body_of_tag5 = match5.group(1)
           
            temp = np.array([body_of_tag1,body_of_tag2,body_of_tag3,body_of_tag4])
            
            x.append(np.append(l[i],temp))
            y.append(body_of_tag5)
            
        i+=1

In [138]:
 # Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
y = le.fit_transform(y)

In [139]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state= 0, train_size = 0.2)

In [140]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [141]:
from sklearn.svm import SVC
#Create a svm Classifier
clf = SVC(kernel='rbf') # Linear Kernel

#Train the model using the training sets
clf.fit(x_train_scaled, y_train)

#Predict the response for test dataset
y_pred = clf.predict(x_test_scaled)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy of SVC: {accuracy}")


Test accuracy of SVC: 0.8201350521792511


In [142]:
best_model = LogisticRegression(solver='newton-cg',C=0.01)
best_model.fit(x_train_scaled, y_train)

# Predict on test data
y_pred = best_model.predict(x_test_scaled)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy of Logistic Regression: {accuracy}")

Test accuracy of Logistic Regression: 0.8192142418661755


In [143]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Train a new model with the best hyperparameters on the combined training and validation sets
Gradient_Boosting_model = GradientBoostingClassifier(n_estimators=10,max_depth=3)
Gradient_Boosting_model.fit(x_train_scaled, y_train)

#Predict the response for test dataset
y_pred = Gradient_Boosting_model.predict(x_test_scaled)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy of GradientBoosting: {accuracy}")


Test accuracy of GradientBoosting: 0.8173726212400245


In [144]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier( random_state = 10 ,max_depth= 4,splitter='random',min_samples_leaf=3)

# Train Decision Tree Classifer
clf = clf.fit(x_train_scaled,y_train)

#Predict the response for test dataset
y_pred = clf.predict(x_test_scaled)
# Model Accuracy, how often is the classifier correct?
print("Test accuracy of DecisionTree:",metrics.accuracy_score(y_test, y_pred))

Test accuracy of DecisionTree: 0.807243707796194
